In [ ]:
# Install necessary packages
! pip install datasets
! pip install -U accelerate
! pip install -U transformers
! pip install seqeval
! pip install evaluate
! pip install pandas

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
# Path to dataset files
path = ""

Mounted at /content/gdrive


In [ ]:
import pandas as pd

# Span
file_path = path + "/argument_span.csv"
df_span = pd.read_csv(file_path, header=0)
df_span = df_span[["text_id", "tokens", "chunk_tags", "split"]]
df_span["tokens"] = df_span.tokens.apply(lambda x: x[1:-1].split(', '))
df_span["tokens"] = df_span["tokens"].apply(lambda x: [i.replace("'", "").strip() for i in x])
df_span["chunk_tags"] = df_span.chunk_tags.apply(lambda x: x[1:-1].split(','))
df_span["chunk_tags"] = df_span["chunk_tags"].apply(lambda x: [int(i, 32) for i in x])
df_span_train = df_span[df_span["split"] == "TRAIN"]
df_span_test = df_span[df_span["split"] == "TEST"]
df_span_val = df_span[df_span["split"] == "VAL"]
df_span_train = df_span_train.drop(columns=["split"], axis=1)
df_span_test = df_span_test.drop(columns=["split"], axis=1)
df_span_val = df_span_val.drop(columns=["split"], axis=1)
df_span_train = df_span_train.reset_index(drop=True)
df_span_test = df_span_test.reset_index(drop=True)
df_span_val = df_span_val.reset_index(drop=True)

# Relations
file_path = path + "/argument_relations.csv"
df_relation = pd.read_csv(file_path, header=0)
df_relation = df_relation[["text_id", "source_tokens", "target_tokens", "labels", "split"]]
df_relation["labels"] = df_relation["labels"].replace("support", 0)
df_relation["labels"] = df_relation["labels"].replace("attack", 1)
df_relation["labels"] = df_relation["labels"].replace("None", 2)
df_relation_train = df_relation[df_relation["split"] == "TRAIN"]
df_relation_test = df_relation[df_relation["split"] == "TEST"]
df_relation_val = df_relation[df_relation["split"] == "VAL"]
df_relation_train = df_relation_train.drop(columns=["split"], axis=1)
df_relation_test = df_relation_test.drop(columns=["split"], axis=1)
df_relation_val = df_relation_val.drop(columns=["split"], axis=1)
df_relation_train = df_relation_train.reset_index(drop=True)
df_relation_test = df_relation_test.reset_index(drop=True)
df_relation_val = df_relation_val.reset_index(drop=True)

# Evaluations
file_path = path + "/text_evaluations.csv"
df_evaluation = pd.read_csv(file_path, header=0)
df_evaluation = df_evaluation[["text_id", "tokens", "final_grade", "split"]]
df_evaluation["final_grade"] = df_evaluation["final_grade"].apply(lambda x: float(x))
df_evaluation_train = df_evaluation[df_evaluation["split"] == "TRAIN"]
df_evaluation_test = df_evaluation[df_evaluation["split"] == "TEST"]
df_evaluation_val = df_evaluation[df_evaluation["split"] == "VAL"]
df_evaluation_train = df_evaluation_train.drop(columns=["split"], axis=1)
df_evaluation_test = df_evaluation_test.drop(columns=["split"], axis=1)
df_evaluation_val = df_evaluation_val.drop(columns=["split"], axis=1)
df_evaluation_train = df_evaluation_train.reset_index(drop=True)
df_evaluation_test = df_evaluation_test.reset_index(drop=True)
df_evaluation_val = df_evaluation_val.reset_index(drop=True)

# Component
df_texts = df_evaluation[["text_id", "tokens"]]
file_path = path + "/argument_components.csv"
df_component = pd.read_csv(file_path, header=0)
df_component = df_component[["text_id", "component_tokens", "labels", "split"]]
df_component = df_component.merge(df_texts, how="left", on="text_id")
df_component.rename(columns={"tokens": "text_tokens"}, inplace=True)
df_component["labels"] = df_component["labels"].replace("MajorClaim", 0)
df_component["labels"] = df_component["labels"].replace("Premise", 1)
df_component_train = df_component[df_component["split"] == "TRAIN"]
df_component_test = df_component[df_component["split"] == "TEST"]
df_component_val = df_component[df_component["split"] == "VAL"]
df_component_train = df_component_train.drop(columns=["split"], axis=1)
df_component_test = df_component_test.drop(columns=["split"], axis=1)
df_component_val = df_component_val.drop(columns=["split"], axis=1)
df_component_train = df_component_train.reset_index(drop=True)
df_component_test = df_component_test.reset_index(drop=True)
df_component_val = df_component_val.reset_index(drop=True)

# Argument Quality
file_path = path + "/argument_quality.csv"
df_argument_quality = pd.read_csv(file_path, header=0)
df_argument_quality = df_argument_quality.drop(columns=["Unnamed: 0"], axis=1)
df_argument_quality_train = df_argument_quality[df_argument_quality["split"] == "TRAIN"]
df_argument_quality_test = df_argument_quality[df_argument_quality["split"] == "TEST"]
df_argument_quality_val = df_argument_quality[df_argument_quality["split"] == "VAL"]
df_argument_quality_train = df_argument_quality_train.drop(columns=["split"], axis=1)
df_argument_quality_test = df_argument_quality_test.drop(columns=["split"], axis=1)
df_argument_quality_val = df_argument_quality_val.drop(columns=["split"], axis=1)
df_argument_quality_train = df_argument_quality_train.reset_index(drop=True)
df_argument_quality_test = df_argument_quality_test.reset_index(drop=True)
df_argument_quality_val = df_argument_quality_val.reset_index(drop=True)

In [ ]:
import evaluate
import datasets
from datasets import Dataset, DatasetDict, Features
from datasets import load_dataset, load_metric

metric = evaluate.load("seqeval")

# **Span Detection**

In [ ]:
# Create Dataset I for span detection

features = Features(
    (
      {
          "text_id": datasets.Value("int32"),
          "tokens": datasets.Sequence(datasets.Value("string")),
          "chunk_tags":datasets.Sequence(
              datasets.features.ClassLabel(
                  names=[
                      "O",
                      "B-ARG",
                      "I-ARG",
                  ]
              )
          )
      }
        )
  )

train_ds = Dataset.from_pandas(df_span_train, features=features)
test_ds = Dataset.from_pandas(df_span_test, features=features)
val_ds = Dataset.from_pandas(df_span_val, features=features)
original_data_span = DatasetDict()
original_data_span["train"] = train_ds
original_data_span["test"] = test_ds
original_data_span["eval"] = val_ds

In [ ]:
task_feature = original_data_span["train"].features["chunk_tags"]
label_names = task_feature.feature.names

In [ ]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

model_checkpoint = "roberta-base"
#model_checkpoint = "bert-base-cased"

from transformers import (
    AutoModelForTokenClassification, AutoTokenizer,
    )

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

model =  AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions

In [ ]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }


def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels


def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], is_split_into_words=True, truncation=True,
    )
    all_labels = examples["chunk_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

labels = original_data_span["train"][0]["chunk_tags"]

In [ ]:
tokenized_training_input = original_data_span.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=original_data_span["train"].column_names,
)

Map:   0%|          | 0/135 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer

filepath = path + ""

args = TrainingArguments(
    output_dir=filepath,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=15,
    learning_rate=2e-5,
    weight_decay=0.01,
    seed=42,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_training_input["train"],
    eval_dataset=tokenized_training_input["eval"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
# Train pre-trained model
trainer.train()

In [ ]:
# Make prediction
trainer.evaluate(tokenized_training_input["test"])

In [ ]:
predictions = trainer.predict(tokenized_training_input["test"])
pred = np.argmax(predictions.predictions, axis=-1)

Generate Evaluation Metrics (Recreate the components to evaluate the performance)

In [ ]:
true_components = list()
for_test = list()
total = 0
for i, labels in enumerate(list(tokenized_training_input["test"])):
  labels_list = [elem for elem in list(labels["labels"]) if elem != -100]
  text_len = len(labels_list)
  former_label = 0
  start = -1
  end = -1
  for j, label in enumerate(labels_list):
    if label == 1 and former_label == 0:
      start = j
    elif label == 1 and former_label in (1,2):
      end = j-1
      component = [i, start, end]
      true_components.append(str(component))
      start = j
      end = -1
    elif label == 0 and former_label in (1,2):
      end = j-1
      component = [i, start, end]
      true_components.append(str(component))
      start = -1
      end = -1
    elif j+1 == text_len and start != -1:
      end = j
      component = [i, start, end]
      true_components.append(str(component))
      start = -1
      end = -1
    elif label == 2:
      continue
    former_label = label
  if start != -1 and end == -1:
    print(f"ERROR: {i}")
    print(labels_list)
    end = text_len - 1
    component = [i, start, end]
    true_components.append(str(component))

  text_components = len(true_components) - total
  for_test.append(text_components)
  total = len(true_components)

In [ ]:
predicted_components = list()
for i, labels in enumerate(list(pred)):
  labels_list = [elem for elem in list(labels) if elem != -100]
  labels_list.pop(0)
  former_label = 0
  start = -1
  end = -1
  for j, label in enumerate(labels_list):
    if label == 1 and former_label == 0:
      start = j
    elif label == 1 and former_label in (1,2):
      end = j-1
      component = [i, start, end]
      predicted_components.append(str(component))
      start = j
      end = -1
    elif label == 0 and former_label in (1,2):
      end = j-1
      component = [i, start, end]
      predicted_components.append(str(component))
      start = -1
      end = -1
    elif label == labels_list[-1] and start != -1:
      end = j
      component = [i, start, end]
      predicted_components.append(str(component))
      start = -1
      end = -1
    former_label = label
  if start != -1 and end == -1:
    print(f"ERRO: teste {i}")
    end = text_len - 1
    component = [i, start, end]
    predicted_components.append(str(component))

In [ ]:
def intersection(lst1, lst2):
  return list(set(lst1) & set(lst2))

# Not considering truncation
correct_answers = intersection(true_components, predicted_components)

In [ ]:
import re
# Restore Components (To TEST THE CODE)
list_of_true_components = list()

for true_component in true_components:

  # Extract the elements of the list from the string
  elements = re.findall(r'\d+', true_component)

  # Convert the elements to integers
  res = [int(x) for x in elements]

  # Get text
  text_num, start, end = res
  text = tokenized_training_input["test"]["input_ids"][text_num]
  component = list()
  for i in range(start+1, end+2):
    component.append(text[i])
  decoded_component = tokenizer.decode(component, skip_special_tokens= True).strip()
  decoded_component = re.sub(r'[^\w]', ' ', decoded_component).replace(" ", "")
  list_of_true_components.append(decoded_component)

df_corrected = df_component_test[["component_tokens", "text_id"]]
#df_span["tokens"] = df_span.tokens.apply(lambda x: x[1:-1].split(', '))
df_corrected['component_tokens'] = df_corrected.component_tokens.apply(lambda x:re.sub(r'[^\w]', ' ', x).strip().replace(" ", ""))

df_correct = df_corrected[df_corrected["component_tokens"].isin(list_of_true_components)]

In [ ]:
# Restore components from predicted answer

list_of_predicted_components = list()

for predicted_component in predicted_components:

  if predicted_component in correct_answers:
    continue

  # Extract the elements of the list from the string
  elements = re.findall(r'\d+', predicted_component)

  # Convert the elements to integers
  res = [int(x) for x in elements]

  # Get text
  text_num, start, end = res
  text_id = df_span_test.iloc[text_num]["text_id"]
  text_tokens = df_texts[df_texts["text_id"] == text_id].iloc[0]["tokens"]

  text = tokenized_training_input["test"]["input_ids"][text_num]
  component = list()
  for i in range(start+1, end+1):
    component.append(text[i])
  decoded_component = tokenizer.decode(component, skip_special_tokens= True).strip()
  list_of_predicted_components.append({
      "text_id": text_id,
      "component_tokens": decoded_component,
      "labels": "premise",
      "text_tokens": text_tokens
  })

print(len(list_of_predicted_components))
df_wrongly_predicted_components = pd.DataFrame(list_of_predicted_components, columns = ["text_id", "component_tokens", "labels", "text_tokens"])

relations_data = list()

for component in list_of_predicted_components:
  text_id = component["text_id"]
  source_text = component["component_tokens"]
  for target_component in list_of_predicted_components:
    if source_text == target_component["component_tokens"] or text_id != target_component["text_id"]:
      continue
    else:
      target_text = target_component["component_tokens"]
      relation_data = dict()
      relation_data["text_id"] = text_id
      relation_data["source_tokens"] = source_text
      relation_data["target_tokens"] = target_text
      relation_data["labels"] = "None"
      relations_data.append(relation_data)

df_wrongly_predicted_relations = pd.DataFrame(relations_data, columns = relations_data[0].keys())

In [ ]:
# Restore Components from correct answers
list_of_correct_components = list()

for correct_component in correct_answers:

  # Extract the elements of the list from the string
  elements = re.findall(r'\d+', correct_component)

  # Convert the elements to integers
  res = [int(x) for x in elements]

  # Get text
  text_num, start, end = res
  text = tokenized_training_input["test"]["input_ids"][text_num]
  component = list()
  for i in range(start+1, end+2):
    component.append(text[i])
  decoded_component = tokenizer.decode(component, skip_special_tokens= True).strip()
  decoded_component = re.sub(r'[^\w]', ' ', decoded_component).replace(" ", "").strip()
  list_of_correct_components.append(decoded_component)

# Get component test dataframe, but only with the components indeed correctly found

df_corrected = df_component_test.copy()
df_corrected['component_tokens'] = df_corrected.component_tokens.apply(lambda x:re.sub(r'[^\w]', ' ', x).strip().replace(" ", ""))

df_corrected = df_corrected[df_corrected["component_tokens"].isin(list_of_correct_components)]
print(df_corrected.shape)

#print(list(set(list_of_correct_components) - set(df_corrected["component_tokens"].to_list())))

index_list = df_corrected.index.values.tolist()
df_component_test_corrected = df_component_test.iloc[index_list]
df_component_test_corrected = df_component_test_corrected.reset_index(drop=True)

# Get relations test dataframe, but only with the components indeed correctly found
df_corrected = df_relation_test.copy()
df_corrected["source_tokens"] = df_corrected.source_tokens.apply(lambda x:re.sub(r'[^\w]', ' ', x).strip().replace(" ", ""))
df_corrected["target_tokens"] = df_corrected.target_tokens.apply(lambda x:re.sub(r'[^\w]', ' ', x).strip().replace(" ", ""))

df_corrected = df_corrected[df_corrected["source_tokens"].isin(list_of_correct_components)]
df_corrected = df_corrected[df_corrected["target_tokens"].isin(list_of_correct_components)]

index_list = df_corrected.index.values.tolist()
df_relationt_test_corrected = df_relation_test.iloc[index_list]
df_relationt_test_corrected = df_relationt_test_corrected.reset_index(drop=True)

In [ ]:
# Properly considers truncation
inter = df_component_test_corrected.shape[0]

precision = len(predicted_components) and inter/len(predicted_components) or 0
recall = len(true_components) and inter/len(true_components) or 0
f1 = precision and recall and (2*precision*recall)/(precision+recall) or 0

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

# **Component Classification**

In [ ]:
# Create Dataset II for component classificantion
features = Features(
    (
      {
          "text_id": datasets.Value("int32"),
          "component_tokens": datasets.Value("string"),
          "labels": datasets.features.ClassLabel(
                  names=[
                      "major claim",
                      "premise",
                  ]
              ),
          "text_tokens": datasets.Value("string"),
      }
        )
  )
train_ds = Dataset.from_pandas(df_component_train, features=features)
test_ds = Dataset.from_pandas(df_component_test, features=features)
corrected_test_ds = Dataset.from_pandas(df_component_test_corrected, features=features)
predicted_test_ds =  Dataset.from_pandas(df_wrongly_predicted_components, features=features)
val_ds = Dataset.from_pandas(df_component_val, features=features)
original_data_component = DatasetDict()
original_data_component["train"] = train_ds
original_data_component["test"] = test_ds
original_data_component["eval"] = val_ds
original_data_component["corrected_test"] = corrected_test_ds
original_data_component["predicted_test"] = predicted_test_ds

In [ ]:
task_feature = original_data_component["train"].features["labels"]
label_names = task_feature.names
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
     DataCollatorWithPadding
    )

model_checkpoint = "roberta-base"
#model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

model_component = AutoModelForSequenceClassification.from_pretrained(
  model_checkpoint, id2label=id2label, label2id=label2id,
)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["component_tokens"], examples["text_tokens"], padding=True, truncation="only_second")

tokenized_training_input = original_data_component.map(preprocess_function, batched=True, remove_columns=["component_tokens", "text_tokens"])

In [ ]:
from torch import nn
import torch
from transformers import Trainer


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([9.0, 1.0]).to("cuda"))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np
import pandas as pd
from transformers import TrainingArguments, Trainer


def compute_metrics_component(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(average="macro", y_true=labels, y_pred=pred)
    precision = precision_score(average="macro", y_true=labels, y_pred=pred)
    f1_macro = f1_score(average="macro", y_true=labels, y_pred=pred )
    f1 = f1_score(average="micro", y_true=labels, y_pred=pred )
    f1_weighted = f1_score(average="weighted", y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1, "f1_macro": f1_macro, "f1_weighted": f1_weighted}


filepath = path + ""

component_args = TrainingArguments(
    output_dir=filepath,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=15,
    learning_rate=2e-5,
    weight_decay=0.01,
    seed=42,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True
)
component_trainer = CustomTrainer(
    model=model_component,
    args=component_args,
    train_dataset=tokenized_training_input["train"],
    eval_dataset=tokenized_training_input["eval"],
    data_collator=data_collator,
    compute_metrics=compute_metrics_component,
)

In [ ]:
# Train pre-trained model
component_trainer.train()

In [ ]:
# Make predictions
component_trainer.evaluate(tokenized_training_input["test"])

In [ ]:
# Corrected metrics
corrected_prediction = component_trainer.evaluate(tokenized_training_input["corrected_test"])
acc = corrected_prediction["eval_accuracy"]
num_correct_components_final = int(acc*inter)

precision = len(predicted_components) and num_correct_components_final/len(predicted_components) or 0
recall = len(true_components) and num_correct_components_final/len(true_components) or 0
f1 = precision and recall and (2*precision*recall)/(precision+recall) or 0

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

# Corrected F1 Macro
sum = 0
predictions = component_trainer.predict(tokenized_training_input["predicted_test"])
pred = np.argmax(predictions.predictions, axis=-1)
labels = ["major claim", "premise"]

for label in [0, 1]:
  corrected_label_prediction = component_trainer.evaluate(tokenized_training_input["corrected_test"].filter(lambda example: example["labels"] == label))
  print(corrected_label_prediction)
  label_acc = corrected_label_prediction["eval_accuracy"]

  num_corrected_label = df_component_test_corrected[df_component_test_corrected["labels"] == labels[label]].shape[0]
  num_correct_components = int(label_acc*num_corrected_label)
  num_test_label = df_component_test[df_component_test["labels"] == labels[label]].shape[0]
  num_predicted_label = list(pred).count(label)

  precision = num_corrected_label and num_correct_components/(num_predicted_label+num_correct_components) or 0
  recall = num_test_label and num_correct_components/num_test_label or 0
  f1_macro = precision and recall and (2*precision*recall)/(precision+recall) or 0
  print(num_correct_components)


  sum = sum + f1_macro

macro = sum/2

print(f"F1-score Macro: {macro}")

# **Relation Classification**

In [ ]:
# Create Dataset III for relation classificantion
features = Features(
    (
      {
          "text_id": datasets.Value("int32"),
          "source_tokens": datasets.Value("string"),
          "target_tokens": datasets.Value("string"),
          "labels": datasets.features.ClassLabel(
                  names=[
                      "support",
                      "attack",
                      "None"
                  ]
              )
      }
        )
  )
train_ds = Dataset.from_pandas(df_relation_train, features=features)
test_ds = Dataset.from_pandas(df_relation_test, features=features)
corrected_test_ds = Dataset.from_pandas(df_relationt_test_corrected, features=features)
predicted_test_ds = Dataset.from_pandas(df_wrongly_predicted_relations, features=features)
val_ds = Dataset.from_pandas(df_relation_val, features=features)
original_data_relation = DatasetDict()
original_data_relation["train"] = train_ds
original_data_relation["test"] = test_ds
original_data_relation["eval"] = val_ds
original_data_relation["corrected_test"] = corrected_test_ds
original_data_relation["predicted_test"] = predicted_test_ds

In [ ]:
task_feature = original_data_relation["train"].features["labels"]
label_names = task_feature.names
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
     DataCollatorWithPadding
    )

model_checkpoint = "roberta-base"
#model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

model_relation = AutoModelForSequenceClassification.from_pretrained(
  model_checkpoint, id2label=id2label, label2id=label2id,
)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["source_tokens"], examples["target_tokens"], padding=True, truncation=True)

tokenized_training_input = original_data_relation.map(preprocess_function, batched=True, remove_columns=["source_tokens", "target_tokens"])

In [ ]:
from torch import nn
import torch
from transformers import Trainer


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([2.0, 1.0, 0.06]).to("cuda"))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np
import pandas as pd
from transformers import TrainingArguments, Trainer


def compute_metrics_relations(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(average="macro", y_true=labels, y_pred=pred)
    precision = precision_score(average="macro", y_true=labels, y_pred=pred)
    f1 = f1_score(average="micro", y_true=labels, y_pred=pred)
    f1_macro = f1_score(average="macro", y_true=labels, y_pred=pred)
    f1_weighted = f1_score(average="weighted", y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1, "f1_macro": f1_macro, "f1_weighted": f1_weighted}

filepath = path + ""

relation_args = TrainingArguments(
    output_dir=filepath,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=15,
    learning_rate=2e-5,
    weight_decay=0.01,
    seed=42,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
)
relation_trainer = CustomTrainer(
    model=model_relation,
    args=relation_args,
    train_dataset=tokenized_training_input["train"],
    eval_dataset=tokenized_training_input["eval"],
    data_collator=data_collator,
    compute_metrics=compute_metrics_relations,
)

In [ ]:
# Train pre-trained model
relation_trainer.train()

In [ ]:
# Make prediction
relation_trainer.evaluate(tokenized_training_input["test"])

In [ ]:
## Corrected metrics
from itertools import combinations

num_possible_correct_relations = df_relationt_test_corrected.shape[0]
num_relations = df_relation_test.shape[0]

# Duas vezes combinacao de total de componentes previstos para cada grafo 2 a 2
components_per_text = dict()
for predicted_component in predicted_components:
  # Extract the elements of the list from the string
  elements = re.findall(r"\d+", predicted_component)

  # Get text
  text_num, start, end = elements

  # Components per text
  if not components_per_text.get(text_num, False):
    components_per_text[text_num] = 1
  else:
    components_per_text[text_num] += 1

total_predicted_relations = 0
for key in components_per_text.keys():

  num_component_per_graph = components_per_text[key]
  list_evaluation_modules = list(range(1,num_component_per_graph+1,1))
  comb = combinations(list_evaluation_modules, 2)
  total_predicted_relations += 2*len(list(comb))

corrected_prediction = relation_trainer.evaluate(tokenized_training_input["corrected_test"])
acc = corrected_prediction["eval_accuracy"]
num_correct_relations = int(acc*num_possible_correct_relations)

precision = total_predicted_relations and num_correct_relations/total_predicted_relations or 0
recall = num_relations and num_correct_relations/num_relations or 0
f1 = precision and recall and (2*precision*recall)/(precision+recall) or 0

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

# Corrected F1 Macro
sum = 0
predictions = relation_trainer.predict(tokenized_training_input["predicted_test"])
pred = np.argmax(predictions.predictions, axis=-1)

for label in [0, 1, 2]:
  corrected_label_prediction = relation_trainer.evaluate(tokenized_training_input["corrected_test"].filter(lambda example: example["labels"] == label))
  label_acc = corrected_label_prediction["eval_accuracy"]

  num_test_label = df_relation_test[df_relation_test["labels"] == label].shape[0]
  num_corrected_label = df_relationt_test_corrected[df_relationt_test_corrected["labels"] == label].shape[0]
  num_correct_relations = int(label_acc*num_corrected_label)
  num_predicted_label = list(pred).count(label)

  precision = num_corrected_label and num_correct_relations/(num_correct_relations+num_predicted_label) or 0
  recall = num_test_label and num_correct_relations/num_test_label or 0
  f1_macro = precision and recall and (2*precision*recall)/(precision+recall) or 0
  sum = sum + f1_macro

macro = sum/3

print(f"F1-score Macro: {macro}")

# **Text Evaluation**

In [ ]:
# Create Dataset IV for text evaluation
features = Features(
    (
      {
          "text_id": datasets.Value("int32"),
          "tokens": datasets.Value("string"),
          "final_grade": datasets.Value("float32")
      }
        )
  )
train_ds = Dataset.from_pandas(df_evaluation_train, features=features)
test_ds = Dataset.from_pandas(df_evaluation_test, features=features)
val_ds = Dataset.from_pandas(df_evaluation_val, features=features)
original_data_evaluation = DatasetDict()
original_data_evaluation["train"] = train_ds
original_data_evaluation["test"] = test_ds
original_data_evaluation["eval"] = val_ds

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from torch.utils.data import DataLoader

model_checkpoint = "roberta-base"
#model_checkpoint = "bert-base-cased"
LEARNING_RATE = 2e-5
MAX_LENGTH = 512
BATCH_SIZE = 8
EPOCHS = 15

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=1)

In [ ]:
# ds = {"train": raw_train_ds, "validation": raw_val_ds, "test": raw_test_ds}

def preprocess_function(examples):
    label = examples["final_grade"]
    examples = tokenizer(examples["tokens"], truncation=True, padding="max_length", max_length=512
                         )

    # Change this to real number
    examples["label"] = float(label)
    return examples

tokenized_training_input = original_data_evaluation.map(preprocess_function, remove_columns=["text_id", "tokens", "final_grade"])

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import cohen_kappa_score
import numpy as np

def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)
    qwk_logits = 100*logits
    qwk_labels = 100*labels
    qwk_logits = qwk_logits.astype(int)
    qwk_labels = qwk_labels.astype(int)

    mse = mean_squared_error(labels, logits)
    qwk = cohen_kappa_score(qwk_labels, qwk_logits, weights="quadratic")

    return {"mse": mse, "qwk": qwk}

In [ ]:
from transformers import TrainingArguments, Trainer

filepath = path + ""

evaluation_training_args = TrainingArguments(
    output_dir=filepath,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="qwk",
    load_best_model_at_end=True,
    weight_decay=0.01,
    greater_is_better=True,
    seed=42
)

trainer = Trainer(
    model=model,
    args=evaluation_training_args,
    train_dataset=tokenized_training_input["train"],
    eval_dataset=tokenized_training_input["eval"],
    compute_metrics=compute_metrics_for_regression
)

trainer.train()

In [ ]:
trainer.eval_dataset = tokenized_training_input["test"]
trainer.evaluate()

# **Argument Quality**

In [ ]:
# Create Dataset V for argument quality evaluation
features = Features(
    (
      {
          "text_id": datasets.Value("int32"),
          "tokens": datasets.Value("string"),
          "arg_quality": datasets.features.ClassLabel(
                  names=[
                      "good",
                      "bad",
                      "ugly"
                  ]
              )
      }
        )
  )
train_ds = Dataset.from_pandas(df_argument_quality_train, features=features)
test_ds = Dataset.from_pandas(df_argument_quality_test, features=features)
val_ds = Dataset.from_pandas(df_argument_quality_val, features=features)
original_data_argument = DatasetDict()
original_data_argument["train"] = train_ds
original_data_argument["test"] = test_ds
original_data_argument["eval"] = val_ds

In [ ]:
task_feature = original_data_argument["train"].features["arg_quality"]
label_names = task_feature.names
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
     DataCollatorWithPadding
    )

#model_checkpoint = "roberta-base"
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

model_arg_quality = AutoModelForSequenceClassification.from_pretrained(
  model_checkpoint, id2label=id2label, label2id=label2id,
)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def preprocess_function(examples):
    label = examples["arg_quality"]
    examples = tokenizer(
        examples["tokens"], truncation=True, padding=True,
    )
    examples["label"] = label
    return examples

tokenized_training_input = original_data_argument.map(preprocess_function, batched=True, remove_columns=["text_id", "tokens", "arg_quality"])

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np
import pandas as pd
from transformers import TrainingArguments, Trainer


def compute_metrics_quality(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(average="macro", y_true=labels, y_pred=pred)
    precision = precision_score(average="macro", y_true=labels, y_pred=pred)
    f1 = f1_score(average="micro", y_true=labels, y_pred=pred)
    f1_macro = f1_score(average="macro", y_true=labels, y_pred=pred)
    f1_weighted = f1_score(average="weighted", y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1, "f1_macro": f1_macro, "f1_weighted": f1_weighted}

filepath = path + "/Modelos/Revisao/Quality_Task/Bert_Quality"

quality_args = TrainingArguments(
    output_dir=filepath,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=15,
    learning_rate=2e-5,
    weight_decay=0.01,
    seed=42,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True
)
quality_trainer = Trainer(
    model=model_arg_quality,
    args=quality_args,
    train_dataset=tokenized_training_input["train"],
    eval_dataset=tokenized_training_input["eval"],
    data_collator=data_collator,
    compute_metrics=compute_metrics_quality,
)

In [ ]:
# Train pre-trained model
quality_trainer.train()

In [ ]:
# Make prediction
quality_trainer.evaluate(tokenized_training_input["test"])